In [2]:
import pandas as pd
from IPython.display import display

In [142]:
import numpy as np
import pandas as pd
from IPython.display import display

# Load the JSON data
url = 'https://hackscfitgpt-default-rtdb.firebaseio.com/data.json'
data = pd.read_json(url, orient='index').T['bodyMeasurements']

# Initialize a list to store the structured data
structured_data = []

# Extract each entry from `bodyMeasurements`
for measurement in data.values:
    if isinstance(measurement, dict):  # Only process if it's a dictionary
        name = measurement.get('name', '')
        units = measurement.get('units', '')
        for entry in measurement.get('data', []):
            structured_data.append({
                'date': entry.get('date', ''),
                'qty': entry.get('qty', ''),
                'name': name,
                'units': units
            })

# Create a DataFrame from the structured list
output_df = pd.DataFrame(structured_data, columns=['date', 'qty', 'name', 'units'])
len(output_df['name'].unique())
display(output_df)

,date,qty,name,units
0,2024-11-02 10:25:00 -0700,116.073381,lean_body_mass,lb
1,2024-11-02 10:25:00 -0700,21.5,body_mass_index,count
2,2024-11-01 19:49:00 -0700,90,walking_heart_rate_average,count/min
3,2024-11-02 12:44:00 -0700,113,walking_heart_rate_average,count/min
4,2024-11-03 10:04:00 -0800,110,walking_heart_rate_average,count/min
...,...,...,...,...
30628,2024-11-09 22:05:00 -0800,4.8,physical_effort,kcal/hr·kg
30629,2024-11-09 22:06:00 -0800,3.2,physical_effort,kcal/hr·kg
30630,2024-11-09 22:07:00 -0800,2.925,physical_effort,kcal/hr·kg
30631,2024-11-09 22:09:00 -0800,3.425,physical_effort,kcal/hr·kg


In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

# Load the JSON data
url = 'https://hackscfitgpt-default-rtdb.firebaseio.com/data.json'
data = pd.read_json(url, orient='index').T['bodyMeasurements']

# Initialize a list to store the structured data
structured_data = []

# Extract each entry from `bodyMeasurements`
for measurement in data.values:
    if isinstance(measurement, dict):  # Only process if it's a dictionary
        name = measurement.get('name', '')
        units = measurement.get('units', '')
        for entry in measurement.get('data', []):
            structured_data.append({
                'date': entry.get('date', ''),
                'qty': entry.get('qty', ''),
                'name': name,
                'units': units
            })

# Create a DataFrame from the structured list
output_df = pd.DataFrame(structured_data, columns=['date', 'qty', 'name', 'units'])

# Convert the 'date' column to datetime
output_df['date'] = pd.to_datetime(output_df['date'], errors='coerce', utc=True)


# Sort the DataFrame by 'name' and 'date' to ensure correct time-based grouping
output_df = output_df.sort_values(by=['name', 'date'])

# Display the DataFrame
display(output_df)

,date,qty,name,units
2816,2024-11-01 14:53:00+00:00,0.344497,active_energy,kcal
2817,2024-11-01 14:54:00+00:00,0.820503,active_energy,kcal
2818,2024-11-02 02:46:00+00:00,8.280599,active_energy,kcal
2819,2024-11-02 02:47:00+00:00,2.369133,active_energy,kcal
2820,2024-11-02 02:48:00+00:00,0.292474,active_energy,kcal
...,...,...,...,...
6223,2024-11-10 04:17:00+00:00,23.622047,walking_step_length,in
6224,2024-11-10 05:01:00+00:00,14.96063,walking_step_length,in
6225,2024-11-10 05:12:00+00:00,21.062992,walking_step_length,in
6226,2024-11-10 05:13:00+00:00,23.622047,walking_step_length,in


In [184]:
# unique_dates = output_df['date'].dt.date.unique()
# 
# # Print all unique dates
# for date in unique_dates:
#     print(date)


2024-11-01
2024-11-02
2024-11-03
2024-11-04
2024-11-05
2024-11-06
2024-11-07
2024-11-08
2024-11-09
2024-11-10


In [21]:
# Group the data by 'name' and 'date' and calculate the sum of 'qty' include the units also include the units
output_df['qty'] = pd.to_numeric(output_df['qty'], errors='coerce')
output_df['units'] = output_df['units'].astype(str)
output_df

# group the data by 'name' and 'date' and calculate the sum of 'qty' and 'units' with 10 days frequency
avg_10_days = output_df.groupby(['name', pd.Grouper(key='date', freq='10D')]).agg({'qty': 'sum', 'units': 'first'}).reset_index()

# drop date column
avg_10_days.drop(columns='date', inplace=True)

# Now divide the 'qty' by 10 and convert it to an integer
avg_10_days['qty'] = (avg_10_days['qty'] / 10).fillna(0).astype(int)

# Display the DataFrame


# CONVERT THE EACH NAME IN THE ROW AS NAME:QTY UNITS
avg_10_days['name'] = avg_10_days['name'] + ': ' + avg_10_days['qty'].astype(str) + '' + avg_10_days['units']
avg_10_days.drop(columns=['qty', 'units'], inplace=True)
avg_10_days.to_csv('ten_days_average.py', index=False)

In [13]:
# Group the data by 'name' and 'date' with a frequency of 10 days and calculate the sum of 'qty'
avg_10_days = output_df.groupby(['name', pd.Grouper(key='date', freq='10D')])['qty'].sum().reset_index()

# drop date column
avg_10_days.drop(columns='date', inplace=True)

avg_10_days['qty'] = pd.to_numeric(avg_10_days['qty'], errors='coerce')

# Now divide the 'qty' by 10 and convert it to an integer
avg_10_days['qty'] = (avg_10_days['qty'] / 10).fillna(0).astype(int)


avg_10_days




,name,qty
0,active_energy,471
1,apple_exercise_time,60
2,apple_stand_hour,5
3,apple_stand_time,103
4,basal_energy_burned,1327
5,blood_oxygen_saturation,48
6,body_fat_percentage,1
7,body_mass_index,2
8,environmental_audio_exposure,616
9,flights_climbed,9


In [118]:
output_df['date'] = pd.to_datetime(output_df['date'], errors='coerce')
output_df['qty'] = pd.to_numeric(output_df['qty'], errors='coerce')

unique_names = output_df['name'].unique()
print("Unique measurement names:", unique_names)

Unique measurement names: ['lean_body_mass' 'body_mass_index' 'walking_heart_rate_average'
 'apple_exercise_time' 'time_in_daylight' 'headphone_audio_exposure'
 'apple_stand_time' 'blood_oxygen_saturation' 'active_energy' 'vo2_max'
 'stair_speed_down' 'six_minute_walking_test_distance' 'stair_speed_up'
 'walking_asymmetry_percentage' 'environmental_audio_exposure'
 'flights_climbed' 'resting_heart_rate' 'heart_rate_variability'
 'walking_double_support_percentage' 'apple_stand_hour'
 'walking_step_length' 'step_count' 'weight_body_mass'
 'body_fat_percentage' 'walking_running_distance' 'walking_speed'
 'basal_energy_burned' 'heart_rate' 'physical_effort']


In [119]:
summary_stats = output_df.groupby('name')['qty'].describe()
summary_stats

,count,mean,std,min,25%,50%,75%,max
name,,,,,,,,
active_energy,2351.0,2.006957,2.715796,0.005382,0.339561,0.800164,2.570950,12.827866
apple_exercise_time,605.0,1.000000,0.055088,0.042667,1.000000,1.000000,1.000000,1.957333
apple_stand_hour,51.0,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
apple_stand_time,1595.0,0.645768,0.312495,0.200000,0.400000,0.600000,1.000000,1.000000
basal_energy_burned,10638.0,1.247446,14.678222,0.059010,1.060050,1.060050,1.066150,1515.000000
blood_oxygen_saturation,5.0,96.400000,3.130495,91.000000,97.000000,97.000000,98.000000,99.000000
body_fat_percentage,1.0,12.400000,NaN,12.400000,12.400000,12.400000,12.400000,12.400000
body_mass_index,1.0,21.500000,NaN,21.500000,21.500000,21.500000,21.500000,21.500000
environmental_audio_exposure,87.0,70.888434,5.157907,60.879743,67.446098,69.882593,73.223385,87.518243


In [120]:
output_df.columns

Index(['date', 'qty', 'name', 'units'], dtype='object')

In [133]:
output_df['date'] = pd.to_datetime(output_df['date']).dt.tz_convert('UTC')
output_df['date'].unique()
output_df.dropna()
output_df.date = output_df.date.dt.date

In [134]:
output_df.date

0        2024-11-02
1        2024-11-02
2        2024-11-02
3        2024-11-02
4               NaT
            ...    
30628           NaT
30629           NaT
30630           NaT
30631           NaT
30632           NaT
Name: date, Length: 30633, dtype: object